In [15]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/cleaned_merged_seasons.csv')
df = df[(df['minutes']>0)&(df['season_x']=='2021-22')]
players = pd.read_csv('../data/2022-23/players_raw.csv')

/var/folders/c_/ynd73f417v37v9qylcby6lzm0000gq/T/ipykernel_9377/1961697655.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/cleaned_merged_seasons.csv')


In [3]:
print(df.columns.values)
df.head()
df[df['name']=='Bernd Leno']['element']

['Unnamed: 0' 'season_x' 'name' 'position' 'team_x' 'assists' 'bonus'
 'bps' 'clean_sheets' 'creativity' 'element' 'fixture' 'goals_conceded'
 'goals_scored' 'ict_index' 'influence' 'kickoff_time' 'minutes'
 'opponent_team' 'opp_team_name' 'own_goals' 'penalties_missed'
 'penalties_saved' 'red_cards' 'round' 'saves' 'selected' 'team_a_score'
 'team_h_score' 'threat' 'total_points' 'transfers_balance' 'transfers_in'
 'transfers_out' 'value' 'was_home' 'yellow_cards' 'GW']


74337    1
74901    1
75478    1
92549    1
Name: element, dtype: int64

In [4]:
players.columns.values
players[players['id']==2]

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,...,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
1,0,0,69,None,None,1,80201,None,NaN,0,...,27,10,0,0,0,0,0.0,2.2,Leno,0


In [5]:
ids_to_dict_21 = pd.read_csv('../data/2021-22/id_dict.csv')
ids_to_dict_22 = pd.read_csv('../data/2022-23/id_dict.csv')
ids_to_dict_21.head()
ids_to_dict_21.columns

Index(['Understat_ID', ' FPL_ID', ' Understat_Name', ' FPL_Name'], dtype='object')

In [6]:
df = df.merge(ids_to_dict_21,
        how='inner',
        left_on='element',
        right_on=' FPL_ID')
players = players.merge(ids_to_dict_22,
             how='inner',
              left_on='id',
             right_on=' FPL_ID')

In [7]:
gb = df.groupby(['Understat_ID','name', 'position']).agg(
                      {'total_points':['mean','std','sum','count']})
gb.columns = ['mean','std','sum','count']
gb = gb.reset_index()
gb

,Understat_ID,name,position,mean,std,sum,count
0,62,Lukas Rupp,MID,1.222222,0.427793,22,18
1,65,Timo Werner,FWD,2.952381,3.293572,62,21
2,76,Fabian Schär,DEF,3.625000,3.609318,87,24
3,87,Joelinton Cássio Apolinário de Lira,FWD,2.294118,2.588195,78,34
4,101,Allan Saint-Maximin,FWD,3.352941,2.706696,114,34
...,...,...,...,...,...,...,...
531,10405,Pelenda Joshua Dasilva,MID,0.750000,1.164965,6,8
532,10408,Luis Díaz,MID,4.923077,3.616203,64,13
533,10424,Daniel Chesters,MID,1.000000,NaN,1,1
534,10525,Tony Springett,MID,1.000000,1.000000,3,3


In [60]:
players.columns.values

costs_and_history = gb.merge(players,
         how='inner',
         left_on='Understat_ID',
         right_on='Understat_ID'
        )
# players[['element','now_cost']]

In [108]:
costs_and_history = costs_and_history[costs_and_history['count']>30]
costs_and_history = costs_and_history.sort_values('position').reset_index(drop=True)

In [109]:
costs = costs_and_history['now_cost']
val   = costs_and_history['mean']
std = costs_and_history['std']


In [110]:
gk = costs_and_history[costs_and_history['position']=='GK']
defense = costs_and_history[costs_and_history['position']=='DEF']
mid = costs_and_history[costs_and_history['position']=='MID']
fwd = costs_and_history[costs_and_history['position']=='FWD']


In [39]:
mid.shape

(169, 77)

In [44]:
costs_and_history.reset_index()
costs_and_history['position'].unique()

array(['DEF', 'FWD', 'GK', 'MID'], dtype=object)

In [120]:
from mip import Model, xsum, maximize, BINARY

# wi = weights
# pi = profit
# c = capacity : total cost (100)


p = list(val.values)
w = list(costs.values)
std_penalty = list(std.values)
c= 1000
I = range(len(w))
I_g = range(len(gk))
I_d= range(len(defense))
I_m= range(len(mid))
I_f= range(len(fwd))

m = Model("knapsack")
# define vector of chosen players
x = [m.add_var(var_type=BINARY) for i in I]

# objective function
m.objective = maximize(xsum((p[i] * x[i])-np.log(std_penalty[i]) for i in I) )
# m.objective = maximize(xsum((p[i] * x[i] /std_penalty[i] ) for i in I) )


# cost constraint
m += xsum(w[i] * x[i] for i in I) <= c

#constraint for number of DEF
m += xsum(x[i] for i in I_d )==5
#constraint for number of FWD 
m += xsum(x[i + len(defense)] for i in I_f )==3
#constraint for number of GK
m += xsum(x[i+ len(fwd)+ len(defense)] for i in I_g )==2
#constraint for number of MID
m += xsum(x[i+ len(gk)+ len(defense)+ len(fwd)] for i in I_m )==5

# exclude players by id
# example lloris = 1
ids_to_exclude = []

m += xsum(x[i] for i in ids_to_exclude) == 0


m.optimize()

selected = [i for i in I if x[i].x >= 0.99]
print("selected items: {} ".format(selected))


# for i in selected:
#     print
costs_and_history.iloc[selected].sort_values('position')[['name','position','mean','std','sum','now_cost']]

Cgl0002I 59 variables fixed
Cgl0004I processed model has 2 rows, 33 columns (33 integer (33 of which binary)) and 66 elements
Coin3009W Conflict graph built in 0.000 seconds, density: 1.538%
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc0038I Initial state - 2 integers unsatisfied sum - 1
Cbc0038I Solution found of -13.893
Cbc0038I Before mini branch and bound, 31 integers at bound fixed and 0 continuous
Cbc0038I Full problem 2 rows 33 columns, reduced to 0 rows 0 columns
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I Round again with cutoff of -14.0203
Cbc0038I Reduced cost fixing fixed 3 variables on major pass 2
Cbc0038I Pass   1: suminf.    0.09263 (2) obj. -14.0203 iterations 2
Cbc0038I Pass   2: suminf.    1.00000 (2) obj. -14.8536 iterations 2
Cbc0038I Pass   3: suminf.    0.53440 (2) obj. -14.0203 iterations 1
Cbc0038I Pass   4: suminf.    0.55475 (2) obj. -14.0203 iterations 8
Cbc0038I Solution found of -14.1504
Cbc0038I

,name,position,mean,std,sum,now_cost
4,João Pedro Cavaco Cancelo,DEF,5.714286,4.362753,200,70
6,Virgil van Dijk,DEF,5.382353,3.559151,183,65
12,Trent Alexander-Arnold,DEF,6.500000,3.860470,208,75
16,Aymeric Laporte,DEF,4.968750,3.551323,159,60
22,Gabriel Magalhães,DEF,4.235294,4.285697,144,50
29,Ivan Toney,FWD,4.281250,4.152220,137,70
32,Armando Broja,FWD,2.903226,2.890846,90,55
33,Allan Saint-Maximin,FWD,3.352941,2.706696,114,65
44,Alisson Ramses Becker,GK,4.971429,2.925403,174,55
45,Aaron Ramsdale,GK,4.030303,3.107298,133,50


generator 5 (MixedIntegerRounding2) - 1 row cuts average 11.0 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is 1
Cbc0014I Cut generator 6 (FlowCover) - 0 row cuts average 0.0 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is -100
Cbc0014I Cut generator 7 (TwoMirCuts) - 5 row cuts average 11.4 elements, 0 column cuts (0 active)  in 0.000 seconds - new frequency is -100
Cbc0001I Search completed - best objective -14.95, took 3 iterations and 0 nodes (0.01 seconds)
Cbc0035I Maximum depth 0, 21 variables fixed on reduced cost
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 5 (0) rows, 92 (0) columns and 184 (0) elements
Clp1000I sum of infeasibilities 2.42851e-09 - average 4.85701e-10, 45 fixed columns
Coin0506I Presolve 5 (0) rows, 43 (-49) columns and 86 (-98) elements
Clp0006I 0  Obj 72.103603 Dual inf 17663.368 (43)

In [118]:
costs_and_history.iloc[selected]['mean'].sum()


14.999632352941177